In [1]:
%matplotlib notebook

import os, sys, tqdm
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread, imsave
import pandas as pd
import napari

sys.path.append(os.path.join('..','utils'))
import step01_segment_cells_and_fluorescence as funcs

from importlib import reload
reload(funcs)

<module 'step01_segment_cells_and_fluorescence' from '..\\utils\\step01_segment_cells_and_fluorescence.py'>

In [2]:
folders = [
#     os.path.join('..','HCR_time_course','hpf08','Panel1','2021-09-20_084304'), # reference for hpf08
#     os.path.join('..','HCR_time_course','hpf08','Panel1','2021-09-20_084954'), # 
#     os.path.join('..','HCR_time_course','hpf08','Panel1','2021-09-20_085804'), # 
    

#     os.path.join('..','HCR_time_course','hpf10','Panel1','2021-09-20_104647'), # reference for hpf10
#     os.path.join('..','HCR_time_course','hpf10','Panel1','2021-09-20_104949'), # 
#     os.path.join('..','HCR_time_course','hpf10','Panel1','2021-09-20_105356'), # 
    
#     os.path.join('..','HCR_time_course','hpf10','Panel2','2021-10-26_072116'),
#     os.path.join('..','HCR_time_course','hpf10','Panel2','2021-10-26_072414'),
    
#     os.path.join('..','HCR_time_course','hpf10','Panel3','2021-10-26_101501'),
#     os.path.join('..','HCR_time_course','hpf10','Panel3','2021-10-26_101721'),
#     os.path.join('..','HCR_time_course','hpf10','Panel3','2021-10-26_102004'),

#     os.path.join('..','HCR_time_course','hpf10','Panel4','2021-10-26_104907'),
#     os.path.join('..','HCR_time_course','hpf10','Panel4','2021-10-26_105128'),


    os.path.join('..','..','HCR_time_course','hpf12','Panel1','2021-09-20_110602'), # reference for hpf12
#     os.path.join('..','HCR_time_course','hpf12','Panel1','2021-09-20_105648'), # 

#     os.path.join('..','HCR_time_course','hpf12','Panel2','2021-10-26_104331'), 

#     os.path.join('..','HCR_time_course','hpf12','Panel3','2021-10-26_110047'),  
#     os.path.join('..','HCR_time_course','hpf12','Panel3','2021-10-26_110714'),  

#     os.path.join('..','HCR_time_course','hpf12','Panel4','2021-10-26_111407'),  
#     os.path.join('..','..','HCR_time_course','hpf12','Panel4','2021-10-26_111929'),  
]

cell_diameter = np.array([3.,10.,10.])
pixel_size = [2.,0.39,0.39]

gene_markers = { 
    'Panel1': ['h2bgfp','hand2','emx3','sox10','sp5l','nkx2.4b'],
    'Panel2': ['h2bgfp','pax2a','meox1','foxa2'],
    'Panel3': ['h2bgfp','ta','aldh1a2','sox17'],
    'Panel4': ['h2bgfp','sox1','prdm1','otx2'],
}
                

In [3]:
index = 0
folder = folders[index]

panel = folder.split(os.sep)[-2]

gene_markers = gene_markers[panel]

h2bFileName = os.path.join(folder, 'tifs','channels_split','h2bgfp.tif')
imgFiles = [os.path.join(folder,'tifs','channels_split',name+'.tif') for name in gene_markers]

In [4]:
imgFiles

['..\\..\\HCR_time_course\\hpf12\\Panel1\\2021-09-20_110602\\tifs\\channels_split\\h2bgfp.tif',
 '..\\..\\HCR_time_course\\hpf12\\Panel1\\2021-09-20_110602\\tifs\\channels_split\\hand2.tif',
 '..\\..\\HCR_time_course\\hpf12\\Panel1\\2021-09-20_110602\\tifs\\channels_split\\emx3.tif',
 '..\\..\\HCR_time_course\\hpf12\\Panel1\\2021-09-20_110602\\tifs\\channels_split\\sox10.tif',
 '..\\..\\HCR_time_course\\hpf12\\Panel1\\2021-09-20_110602\\tifs\\channels_split\\sp5l.tif',
 '..\\..\\HCR_time_course\\hpf12\\Panel1\\2021-09-20_110602\\tifs\\channels_split\\nkx2.4b.tif']

In [5]:
imgs = [imread(f) for f in tqdm.tqdm(imgFiles)]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:26<00:00,  4.40s/it]


In [6]:
fluos_df = pd.read_csv(os.path.join(folder, 'cells_segmented','cell_fluo.csv'))
num_locs_cells = len(fluos_df)

In [9]:
from napari_animation import Animation

anisotropy = pixel_size[0]/pixel_size[1]
downsample = 2

df_napari = pd.DataFrame({
    'z': fluos_df.z*anisotropy,
    'y': fluos_df.y,
    'x': fluos_df.x
})

props = {i: list(fluos_df[i]) for i in fluos_df.keys()}

viewer = napari.Viewer()
colormaps = ['gray','green','cyan','magenta','yellow','red']

j=0
image_l = [0 for i in imgs]
image_l[j] = viewer.add_image(imgs[0][:,::downsample,::downsample], scale=(anisotropy,downsample,downsample), 
                 name=gene_markers[j], colormap=colormaps[j], blending='additive',
                 contrast_limits=(100,np.max(imgs[0][:,::downsample,::downsample])/2.))
j+=1
for img in tqdm.tqdm(imgs[1:]):
    image_l[j] = viewer.add_image(img[:,::downsample,::downsample], scale=(anisotropy,downsample,downsample), 
                     name=gene_markers[j], colormap=colormaps[j], blending='additive',
                     contrast_limits=(0,np.max(img[:,::downsample,::downsample])/10.))
    j+=1

p = viewer.add_points(df_napari[['z','y','x']].to_numpy(), 
                  properties = props, size=20,
                  name='cells', opacity=1, scale=(1,1,1), n_dimensional=True, face_color='red')

animation = Animation(viewer)
viewer.update_console({'animation': animation})

animation.capture_keyframe()

viewer.dims.current_step = (1950, 0, 0)
animation.capture_keyframe(steps=120)

viewer.dims.ndisplay = 3
p.opacity = 0.5
p.size = 10

viewer.camera.angles = (0, 0, 0)
p.face_color = 'red'
animation.capture_keyframe()
viewer.camera.angles = (0, 90, 0)
animation.capture_keyframe(steps=30)
viewer.camera.angles = (0, 180, 0)
animation.capture_keyframe(steps=30)
viewer.camera.angles = (0, 270, 0)
animation.capture_keyframe(steps=30)
viewer.camera.angles = (0, 360, 0)
animation.capture_keyframe(steps=30)

# for j in range(len(image_l)):
#     image_l[j].visible=False
    
# for k,c  in zip(
#     ['emx3', 'hand2', 'sox10', 'sp5l', 'nkx2.4b'],
#     ['green','cyan','magenta','yellow','red']
# ):
#     viewer.camera.angles = (0, 0, 0)
#     p.face_color = k
#     p.face_colormap = c
#     animation.capture_keyframe()
#     viewer.camera.angles = (0, 90, 0)
#     animation.capture_keyframe(steps=30)
#     viewer.camera.angles = (0, 180, 0)
#     animation.capture_keyframe(steps=30)
#     viewer.camera.angles = (0, 270, 0)
#     animation.capture_keyframe(steps=30)
#     viewer.camera.angles = (0, 360, 0)
#     animation.capture_keyframe(steps=30)
    
animation.animate('demo2D.mov', canvas_only=False)
# napari.run()

C:\Users\nicol\anaconda3\envs\scaling\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = canvas_size / scale
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.36it/s]


Rendering frame  1 of 256


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1920, 1017) to (1920, 1024) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Rendering frame  2 of 256
Rendering frame  3 of 256
Rendering frame  4 of 256
Rendering frame  5 of 256
Rendering frame  6 of 256
Rendering frame  7 of 256
Rendering frame  8 of 256
Rendering frame  9 of 256
Rendering frame  10 of 256
Rendering frame  11 of 256
Rendering frame  12 of 256
Rendering frame  13 of 256
Rendering frame  14 of 256
Rendering frame  15 of 256
Rendering frame  16 of 256
Rendering frame  17 of 256
Rendering frame  18 of 256
Rendering frame  19 of 256
Rendering frame  20 of 256
Rendering frame  21 of 256
Rendering frame  22 of 256
Rendering frame  23 of 256
Rendering frame  24 of 256
Rendering frame  25 of 256
Rendering frame  26 of 256
Rendering frame  27 of 256
Rendering frame  28 of 256
Rendering frame  29 of 256
Rendering frame  30 of 256
Rendering frame  31 of 256
Rendering frame  32 of 256
Rendering frame  33 of 256
Rendering frame  34 of 256
Rendering frame  35 of 256
Rendering frame  36 of 256
Rendering frame  37 of 256
Rendering frame  38 of 256
Rendering